In [1]:
import json
import os
import random
import pyperclip
import hashlib
SRandInt = random.SystemRandom().randint
def getTextFromClipboard():
    return pyperclip.paste()

def pasteTextToClipboard(text):
    pyperclip.copy(text)
    return True

def dec_to_base(num,base=None, special_chars=False):  #Maximum base - 36
    new_num    = ""
    char_set_2  =['!@#$%^&*()_-;:<>?/\|', 'abcdefghijklmnopqrstuvwxyz', 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', '0123456789']
    char_set    = char_set_2[0:] if special_chars else char_set_2[1:]
    chrs        =''.join(char_set)
    if base is None:
        base    = len(chrs)
    while num>0:
        dig     = int(num%base)
        new_num = chrs[dig % len(chrs)] + new_num
        num     //= base
    new_num = new_num[::-1]  #To reverse the string
    return new_num

def getHash(string):
    return int(int(hashlib.sha256(string.encode('utf-8')).hexdigest(), 16)**2)

dec_to_base(getHash('hello'))

'CkOpAWP9gYlDIQnFyplt6ZchKD7bV3xSigCwNKsSOcXTLbgTyTsZGLhQR2XiCvEOTXrHvJHqcIIfQl8IKPbLZb'

In [2]:
def add_passDict_to_passFile(passDict, passFile):
    index = max([int(k) for k in passFile['passwords'].keys()]) + 1 if len(passFile['passwords'].keys()) > 0 else 1
    passFile['passwords'][index] = passDict
    return passFile

def get_passFile(fileName='pass.json'):
    import json
    passFile = None
    seed=''.join([str(SRandInt(0, 9)) for I in range(15)])
    if not os.path.isfile(fileName):
        with open(fileName, 'w') as f:
            json.dump({'seed': seed, 'passwords': {}}, f)
    with open(fileName, 'r') as f:
        passFile = json.load(f)
    return passFile

def DeletePassFile(fileName='pass.json'):
    if os.path.isfile(fileName):
        os.remove(fileName)
        return True
    else:
        return False

def add_to_passFile(passDicts=[], fileName='pass.json'):
    passFile        = get_passFile(fileName)    
    for passDict in passDicts:
        passFile = add_passDict_to_passFile(passDict, passFile)

    with open(fileName, 'w') as f:
        json.dump(passFile, f)

def create_pwd_dict(username, website, length=14):
    return {
        'username': username,
        'website': website,
        'length': length
    }



def get_password(index, passFile, website=None, username=None):
    if type(index) is int:
        index = str(index)
    username, website, seed = passFile['passwords'][index]['username'], passFile['passwords'][index]['website'], passFile['seed']
    hStr = username + website + str(index) + str(getHash(seed)) + seed + seed[::-1]
    return dec_to_base(getHash(hStr), special_chars=True)[:passFile['passwords'][index]['length']]

def passFile_to_list(passFile, website=None):    
    if website is None:
        return [[passFile['passwords'][k]['website'] , passFile['passwords'][k]['username'],get_password(k, passFile)] for k in passFile['passwords'].keys()]
    l = passFile_to_list(passFile)
    return [i for i in l if website in i[0]]

def get_password_for_website(website, passFile=get_passFile()):
    return passFile_to_list(passFile, website)


In [3]:
def AddPassword(username, website, length=14):
    add_to_passFile(passDicts=[create_pwd_dict(username, website, length)])

def GetPasswords(website=None, username=None, passFile=None):
    if website is None:
        website = input('Website: ')
        if website.strip() == '':
            website = getTextFromClipboard()
    if passFile is None:
        passFile = get_passFile()
    pwd = get_password_for_website(website, passFile)
    if pwd is None:
        print('No passwords found for ' + website)
        # return -1
    elif len(pwd) == 1:
        print('Copied to clipboard!')
        pasteTextToClipboard(pwd[0][2])
        # return 0
    else:
        print('Multiple passwords found for ' + website)
        print('Specify Username:')
        if username is None:
            username = input('Username: ')
            if username.strip() == '':
                username = getTextFromClipboard()
        l = [i for i in pwd if i[1] == username]
        if len(l) == 1:
            print('Copied to clipboard!')
            pasteTextToClipboard(l[0][2])
        elif len(l) == 0:
            print('No passwords found for ' + website + ' with username ' + username)
        else:
            print('Multiple passwords found for ' + website + ' and username ' + username)
            print('Copying most recent password')
            pasteTextToClipboard(l[-1][2])

In [4]:

AddPassword(username='user1', website='github.com')
AddPassword(username='user2', website='google.com')
AddPassword(username='user3', website='facebook.com')
passFile_to_list(get_passFile())

[['github.com', 'user1', '^cc7ywol)UX_E3'],
 ['google.com', 'user2', '#*C#t#;eexxAnN'],
 ['facebook.com', 'user3', ')&hFgMtgRRAV1t']]

In [5]:
GetPasswords(website='google.com')

Copied to clipboard!


In [6]:
get_password_for_website('google.com')

[]

In [7]:
# DeletePassFile('pass.json')